In [ ]:
# !docker run -p 9000:9000 -p 9001:9001 \
#   -e MINIO_ROOT_USER=minio \
#   -e MINIO_ROOT_PASSWORD=minio123 \
#   -v $(pwd)/minio_data:/data \
#   minio/minio server /data --console-address ":9001"

docker: Error response from daemon: create $(pwd)/minio_data: "$(pwd)/minio_data" includes invalid characters for a local volume name, only "[a-zA-Z0-9][a-zA-Z0-9_.-]" are allowed. If you intended to pass a host directory, use absolute path

Run 'docker run --help' for more information


In [2]:
# !pip install minio

from minio import Minio
from pathlib import Path


MINIO_ENDPOINT = "127.0.0.1:9000"   
MINIO_USER     = "minio"
MINIO_PASS     = "minio123"
BUCKET         = "model-registry"

client = Minio(MINIO_ENDPOINT, access_key=MINIO_USER, secret_key=MINIO_PASS, secure=False)

if not client.bucket_exists(BUCKET):
    client.make_bucket(BUCKET)

root = Path("model_registry")  
uploaded = []
for p in root.rglob("*"):
    if p.is_file():
        obj = p.relative_to(root).as_posix()   
        client.fput_object(BUCKET, obj, str(p))
        uploaded.append(obj)

print(f"Uploaded {len(uploaded)} objects.")
uploaded[:10]  # show a few


Uploaded 8 objects.


['model1/model.pt',
 'model1/model.joblib',
 'model3/model.pt',
 'model3/model.joblib',
 'model4/model.pt',
 'model4/model.joblib',
 'model2/model.pt',
 'model2/model.joblib']

In [3]:
!cd milestone2/server
!docker build -t id_card:latest .

zsh:cd:1: no such file or directory: milestone2/server
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.0s (2/2) FINISHED                           docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 2B                                         0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
ERROR: failed to solve: failed to read dockerfile: open /var/lib/docker/tmp/buildkit-mount3347318443/Dockerfile: no such file or directory


In [4]:
!cd router
!docker build -t router:latest .

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 DONE 0.0s

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2B done
#1 DONE 0.0s
ERROR: failed to build: failed to solve: failed to read dockerfile: open Dockerfile: no such file or directory


In [3]:
!docker run -d --name model2_a -p 50051:50051 \
  -e VARIANT=model2 \
  -e MINIO_ENDPOINT=host.docker.internal:9000 \
  -e MINIO_USER=minio -e MINIO_PASS=minio123 \
  -e REGISTRY_BUCKET=model-registry \
  id_card:latest


!docker run -d --name model4_a -p 50052:50051 \
  -e VARIANT=model4 \
  -e MINIO_ENDPOINT=host.docker.internal:9000 \
  -e MINIO_USER=minio -e MINIO_PASS=minio123 \
  -e REGISTRY_BUCKET=model-registry \
  id_card:latest

!docker run -d --name model2_b -p 50061:50051 \
  -e VARIANT=model2 \
  -e MINIO_ENDPOINT=host.docker.internal:9000 \
  -e MINIO_USER=minio -e MINIO_PASS=minio123 \
  -e REGISTRY_BUCKET=model-registry \
  id_card:latest


6a561da1fc30f694b089b09feb56c2c28638cef78a4051f325bfbef82e35426e
e1f4b2e76694a94753b72762376e766eaf7dc03f8145a4636f365eba9e430206
7dcba2d7156f154485685ca139b5c7b1f443988c19dfe23c3d634d675f9462fd


In [4]:
!docker run -d --name router -p 50050:50050 \
  -e BACKENDS="model2=host.docker.internal:50051;host.docker.internal:50061,model4=host.docker.internal:50052" \
  router:latest


f9797d4ce7581c3f1ecdc206592b8930a09a8ddd0fc443651bbe19a2368d5198


In [14]:
import grpc
import cv2
import numpy as np

# ensure we can import the stubs
import sys
sys.path.append("server")

import inference_pb2 as pb
import inference_pb2_grpc as pbg


ROUTER_ADDR = "localhost:50050"

# open GRPC channel to router
channel = grpc.insecure_channel(ROUTER_ADDR)
stub = pbg.InferenceStub(channel)

def predict_image(img_path):
    # read the image
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image not found: {img_path}")

 
    ok, enc = cv2.imencode(".jpg", img)
    img_bytes = enc.tobytes()


    req = pb.PredictRequest(image=img_bytes)

    # call using future to capture metadata (which worker served)
    call = stub.Predict.future(req)
    resp = call.result()
    trailers = {k: v for k, v in call.trailing_metadata()}

    print("Request:", img_path)
    print("Prediction:", resp.y)
    print(f"Inference time: {resp.infer_ms:.2f} ms")
    print("Served by model:", trailers.get("x-variant"))
    print("Backend endpoint:", trailers.get("x-target"))

    return resp


In [16]:
predict_image("sample.jpg")

Request: sample.jpg
Prediction: 32.0
Inference time: 24.64 ms
Served by model: model2
Backend endpoint: host.docker.internal:50051


y: 32
infer_ms: 24.637541999254609

In [17]:
predict_image("sample.jpg")
predict_image("sample.jpg")
predict_image("sample.jpg")
predict_image("sample.jpg")
predict_image("sample.jpg")
predict_image("sample.jpg")

Request: sample.jpg
Prediction: 32.0
Inference time: 23.72 ms
Served by model: model4
Backend endpoint: host.docker.internal:50052
Request: sample.jpg
Prediction: 32.0
Inference time: 23.69 ms
Served by model: model2
Backend endpoint: host.docker.internal:50051
Request: sample.jpg
Prediction: 32.0
Inference time: 24.19 ms
Served by model: model4
Backend endpoint: host.docker.internal:50052
Request: sample.jpg
Prediction: 32.0
Inference time: 24.11 ms
Served by model: model4
Backend endpoint: host.docker.internal:50052
Request: sample.jpg
Prediction: 32.0
Inference time: 22.93 ms
Served by model: model4
Backend endpoint: host.docker.internal:50052
Request: sample.jpg
Prediction: 32.0
Inference time: 22.81 ms
Served by model: model4
Backend endpoint: host.docker.internal:50052


y: 32
infer_ms: 22.806416998719214